<a href="https://colab.research.google.com/github/DENGCPU/TensorFlow-Study/blob/main/%E5%AE%98%E7%BD%91demo3_%E4%BD%BF%E7%94%A8TF_Hub%E5%AF%B9%E7%94%B5%E5%BD%B1%E8%AF%84%E8%AE%BA%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 使用TensorFlow Hub和keras进行迁移学习
# 对IMDB的电影评论数据进行分类：25000条训练，25000条测试

In [2]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print('Version:',tf.__version__)
print('Eager mode:', tf.executing_eagerly())
print('Hub version:', hub.__version__)
print('GPU is','available' if tf.config.list_physical_devices('GPU') else 'NOT AVALIABLE')


Version: 2.15.0
Eager mode: True
Hub version: 0.16.1
GPU is available


In [3]:
# 下载IMDB数据集
# 将训练集以6:4切分，15000条训练数据、10000条验证集、25000条测试数据
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]','train[60%:]','test'),
    # 监督学习方式提供，句子有标签
    as_supervised=True
)
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

train_examples_batch

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKDGSOA/imdb_reviews-train.tfrecord…

...

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKDGSOA/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteKDGSOA/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [4]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [7]:
# 构建模型
# 使用Tensorflow Hub的预训练文本嵌入向量模型google/nnlm-en-dim50/2
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
# 训练模型的Keras层
hub_layer = hub.KerasLayer(embedding,input_shape=[],dtype=tf.string,trainable=True)
# 在几个输入样本中进行尝试
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423194 , -0.01190171,  0.06337537,  0.0686297 , -0.16776839,
        -0.10581177,  0.168653  , -0.04998823, -0.31148052,  0.07910344,
         0.15442258,  0.01488661,  0.03930155,  0.19772716, -0.12215477,
        -0.04120982, -0.27041087, -0.21922147,  0.26517656, -0.80739075,
         0.25833526, -0.31004202,  0.2868321 ,  0.19433866, -0.29036498,
         0.0386285 , -0.78444123, -0.04793238,  0.41102988, -0.36388886,
        -0.58034706,  0.30269453,  0.36308962, -0.15227163, -0.4439151 ,
         0.19462997,  0.19528405,  0.05666233,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201319 , -0.04418665, -0.08550781,
        -0.55847436, -0.2333639 , -0.20782956, -0.03543065, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862677,  0.7753425 , -0.07667087,
        -0.15752274,  0.01872334, -0.08169781, -0.3521876 ,  0.46373403,
        -0.08492758,  0.07166861, -0.00670818,  0.12686071, -0.19326551,
 

In [6]:
# 构建完整模型
model = tf.keras.Sequential()
# Keras层，将句子映射到嵌入向量，生成的维度是(num_examples, embedding_dimension)
model.add(hub_layer)
# 添加全连接层，relu是激活函数f(x)=max(x,0), 输出一个16个元素的向量
model.add(tf.keras.layers.Dense(16,activation='relu'))
# 最后一层与单个输出节点紧密相连
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48191433 (183.84 MB)
Trainable params: 48191433 (183.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# 编译模型, 模型输出logits(具有线性激活的单一单元层)
# 使用优化器和损失函数
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
# 训练模型
# 在包含512个样本的mini-batch上进行10个周期的训练，检测在验证集的10000个样本上的损失和准确率
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 9s 222ms/step - loss: 0.6958 - accuracy: 0.5011 - val_loss: 0.6418 - val_accuracy: 0.5452
Epoch 2/10
30/30 [==============================] - 6s 200ms/step - loss: 0.5757 - accuracy: 0.6367 - val_loss: 0.5393 - val_accuracy: 0.7095
Epoch 3/10
30/30 [==============================] - 7s 222ms/step - loss: 0.4485 - accuracy: 0.7947 - val_loss: 0.4336 - val_accuracy: 0.7972
Epoch 4/10
30/30 [==============================] - 6s 189ms/step - loss: 0.3227 - accuracy: 0.8729 - val_loss: 0.3602 - val_accuracy: 0.8415
Epoch 5/10
30/30 [==============================] - 8s 259ms/step - loss: 0.2338 - accuracy: 0.9155 - val_loss: 0.3248 - val_accuracy: 0.8549
Epoch 6/10
30/30 [==============================] - 5s 168ms/step - loss: 0.1724 - accuracy: 0.9433 - val_loss: 0.3103 - val_accuracy: 0.8610
Epoch 7/10
30/30 [==============================] - 6s 195ms/step - loss: 0.1272 - accuracy: 0.9635 - val_loss: 0.3063 - val_accuracy: 0.8657
Epoch 

In [10]:
# 评估模型
results = model.evaluate(test_data.batch(512),verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name,value))

49/49 - 2s - loss: 0.3549 - accuracy: 0.8512 - 2s/epoch - 32ms/step
loss: 0.355
accuracy: 0.851
